<a href="https://colab.research.google.com/github/ahmeddtarekk7769/security-tools/blob/main/Legacy_Cipher_Access_Control_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import random
import sys
import re # Added for password validation
from typing import Dict, Tuple, Optional, Callable, Any

# --- CRITICAL SECURITY WARNING ---
# The Playfair and Vigenère Ciphers are included for historical/educational context ONLY.
# They MUST NEVER be used for real password storage due to being trivial to break.
# Argon2 is the industry standard for secure password hashing.

# External dependency needed: pip install argon2-cffi
try:
    from argon2 import PasswordHasher
    from argon2.exceptions import VerifyMismatchError
    # Configure the Argon2 hasher (default settings are usually robust)
    PH = PasswordHasher()
except ImportError:
    print("WARNING: Argon2 library not found. Run 'pip install argon2-cffi' to enable secure hashing.")
    class PasswordHasher:
        def hash(self, password): return f"MOCKED_HASH:{password}"
        def verify(self, hash, password): return hash == f"MOCKED_HASH:{password}"
        def check_needs_rehash(self, hash): return False
    PH = PasswordHasher()


# --- Configuration and Legacy Ciphers Keys ---

PLAYFAIR_KEY = "CRYPTOGRAPHYKEY"
VIGENERE_KEY = "SECRETWORD"

# --- 1. LEGACY CIPHER IMPLEMENTATIONS ---

class PlayfairCipher:
    """Implements the Playfair Cipher core logic."""
    def __init__(self, key: str):
        self.matrix = self._prepare_key_matrix(key)
        self.char_to_coords = self._build_coord_map()

    def _prepare_key_matrix(self, key: str) -> list[list[str]]:
        key = key.upper().replace('J', 'I')
        key_chars = []
        for char in key:
            if char.isalpha() and char not in key_chars:
                key_chars.append(char)
        alphabet = [c for c in string.ascii_uppercase if c != 'J' and c not in key_chars]
        all_chars = key_chars + alphabet
        matrix = []
        for i in range(5):
            matrix.append(all_chars[i*5:(i+1)*5])
        return matrix

    def _build_coord_map(self) -> Dict[str, Tuple[int, int]]:
        coords = {}
        for r in range(5):
            for c in range(5):
                coords[self.matrix[r][c]] = (r, c)
        return coords

    def _process_text(self, text: str) -> str:
        text = text.upper().replace('J', 'I')
        processed = []
        i = 0
        while i < len(text):
            char1 = text[i]
            if not char1.isalpha():
                i += 1
                continue
            if i == len(text) - 1:
                processed.append(char1)
                processed.append('X')
                break
            char2 = text[i+1]
            if not char2.isalpha():
                i += 1
                processed.append(char1)
                continue
            if char1 == char2:
                processed.append(char1)
                processed.append('X')
                i += 1
            else:
                processed.append(char1)
                processed.append(char2)
                i += 2
        if len(processed) % 2 != 0:
            processed.append('X')
        return "".join(processed)

    def _cipher_pair(self, char1: str, char2: str, mode: str = 'encrypt') -> Tuple[str, str]:
        r1, c1 = self.char_to_coords[char1]
        r2, c2 = self.char_to_coords[char2]
        direction = 1 if mode == 'encrypt' else -1
        if r1 == r2:
            new_c1 = (c1 + direction) % 5
            new_c2 = (c2 + direction) % 5
            return self.matrix[r1][new_c1], self.matrix[r2][new_c2]
        elif c1 == c2:
            new_r1 = (r1 + direction) % 5
            new_r2 = (r2 + direction) % 5
            return self.matrix[new_r1][c1], self.matrix[new_r2][c2]
        else:
            return self.matrix[r1][c2], self.matrix[r2][c1]

    def encrypt(self, plaintext: str) -> str:
        processed_text = self._process_text(plaintext)
        ciphertext = []
        for i in range(0, len(processed_text), 2):
            char1 = processed_text[i]
            char2 = processed_text[i+1]
            c1, c2 = self._cipher_pair(char1, char2, 'encrypt')
            ciphertext.append(c1)
            ciphertext.append(c2)
        return "".join(ciphertext)

    def decrypt(self, ciphertext: str) -> str:
        ciphertext = ciphertext.upper().replace(' ', '')
        plaintext = []
        for i in range(0, len(ciphertext), 2):
            char1 = ciphertext[i]
            char2 = ciphertext[i+1]
            p1, p2 = self._cipher_pair(char1, char2, 'decrypt')
            plaintext.append(p1)
            plaintext.append(p2)
        return "".join(plaintext)


class VigenereCipher:
    """Implements the Vigenère Cipher core logic."""
    def __init__(self, key: str):
        self.key = key.upper()
        self.alphabet = string.ascii_uppercase
        self.base = len(self.alphabet)

    def _process_char(self, char: str, key_char: str, mode: str = 'encrypt') -> str:
        if not char.isalpha(): return char
        char_index = self.alphabet.find(char.upper())
        key_index = self.alphabet.find(key_char.upper())
        if mode == 'encrypt':
            new_index = (char_index + key_index) % self.base
        else: # decrypt
            new_index = (char_index - key_index) % self.base
        return self.alphabet[new_index]

    def encrypt(self, plaintext: str) -> str:
        plaintext = plaintext.upper()
        ciphertext = []
        key_index = 0
        for char in plaintext:
            if char.isalpha():
                key_char = self.key[key_index % len(self.key)]
                ciphertext.append(self._process_char(char, key_char, 'encrypt'))
                key_index += 1
            else:
                ciphertext.append(char)
        return "".join(ciphertext)

    def decrypt(self, ciphertext: str) -> str:
        ciphertext = ciphertext.upper()
        plaintext = []
        key_index = 0
        for char in ciphertext:
            if char.isalpha():
                key_char = self.key[key_index % len(self.key)]
                plaintext.append(self._process_char(char, key_char, 'decrypt'))
                key_index += 1
            else:
                plaintext.append(char)
        return "".join(plaintext)


# --- 2. HASH/VERIFY WRAPPER CLASSES ---

# Initialize core cipher instances
playfair_cipher = PlayfairCipher(PLAYFAIR_KEY)
vigenere_cipher = VigenereCipher(VIGENERE_KEY)

class LegacyCipherWrapper:
    """Base class for legacy ciphers (INSECURE)."""
    def __init__(self, cipher_instance):
        self.cipher = cipher_instance
        self.security_level = "INSECURE"

    def hash(self, password: str) -> str:
        """Encrypts the password for storage (reversible)."""
        return self.cipher.encrypt(password)

    def verify(self, stored_cipher_text: str, submitted_password: str) -> bool:
        """Checks if the submitted password matches the stored ciphertext."""
        raise NotImplementedError

class PlayfairWrapper(LegacyCipherWrapper):
    """Playfair implementation of the verification logic."""
    def __init__(self):
        super().__init__(playfair_cipher)

    def verify(self, stored_cipher_text: str, submitted_password: str) -> bool:
        submitted_cipher_text = self.cipher.encrypt(submitted_password)
        return submitted_cipher_text == stored_cipher_text

class VigenereWrapper(LegacyCipherWrapper):
    """Vigenère implementation of the verification logic."""
    def __init__(self):
        super().__init__(vigenere_cipher)

    def verify(self, stored_cipher_text: str, submitted_password: str) -> bool:
        decrypted_stored_password = self.cipher.decrypt(stored_cipher_text)
        return submitted_password.upper() == decrypted_stored_password

class Argon2Wrapper:
    """Argon2 implementation for secure, one-way hashing (SECURE)."""
    def __init__(self):
        self.security_level = "SECURE"

    def hash(self, password: str) -> str:
        """Hashes the password using Argon2."""
        return PH.hash(password)

    def verify(self, stored_hash: str, submitted_password: str) -> bool:
        """Verifies the password against the stored hash."""
        try:
            PH.verify(stored_hash, submitted_password)
            if PH.check_needs_rehash(stored_hash):
                print(f"[AUTH INFO] Hash needs re-hashing.")
            return True
        except VerifyMismatchError:
            return False
        except Exception as e:
            print(f"[AUTH ERROR] Argon2 verification failed: {e}")
            return False

# --- 3. METHOD MAP ---

AUTH_METHODS: Dict[str, Any] = {
    'argon2': Argon2Wrapper(),
    'playfair': PlayfairWrapper(),
    'vigenere': VigenereWrapper(),
}


# --- 4. ACCESS CONTROL SYSTEM ---

# Simulated User Database: Maps username to {method_name: stored_data}
USER_DB: Dict[str, Dict[str, str]] = {}


def register_user(username: str, stored_data: str, use_method: str) -> bool:
    """Simulates user registration, storing the specific data provided."""
    method_key = use_method.lower()

    if method_key not in AUTH_METHODS:
        print(f"Unknown method type: {use_method}")
        return False

    # Store the new method/data pair
    if username not in USER_DB:
        USER_DB[username] = {}

    USER_DB[username][method_key] = stored_data

    auth_handler = AUTH_METHODS[method_key]
    print(f"[REGISTER] {username} registered/updated using {use_method.title()} ({auth_handler.security_level}).")
    if method_key == 'argon2':
         print(f"  Hash (SECURE): {stored_data[:40]}...")
    else:
         print(f"  Ciphertext (INSECURE): {stored_data}")

    return True

def authenticate_user(username: str, submitted_password: str) -> bool:
    """Simulates user authentication against stored credentials."""
    user_methods = USER_DB[username]
    overall_success = False

    # We no longer print detailed checks, just the result
    for method_key, stored_data in user_methods.items():
        auth_handler = AUTH_METHODS.get(method_key)

        if auth_handler and auth_handler.verify(stored_data, submitted_password):
            overall_success = True
            break # Since we only save Argon2, one successful check is enough

    if overall_success:
        print("user found")
    else:
        print("Authentication failed.")

    return overall_success


def validate_password(password: str) -> bool:
    """Checks if the password meets the complexity requirements."""
    if len(password) < 8:
        print("ERROR: Password must be at least 8 characters long.")
        return False
    # Check for at least one capital letter
    if not re.search(r"[A-Z]", password):
        print("ERROR: Password must contain at least one capital letter (A-Z).")
        return False
    # Check for at least one small letter
    if not re.search(r"[a-z]", password):
        print("ERROR: Password must contain at least one small letter (a-z).")
        return False
    # Check for at least one number
    if not re.search(r"[0-9]", password):
        print("ERROR: Password must contain at least one number (0-9).")
        return False
    # Check for at least one special character (using any character that is not alphanumeric)
    if not re.search(r"[^a-zA-Z0-9]", password):
        print("ERROR: Password must contain at least one special character (e.g., !, @, #, $).")
        return False
    return True


# --- 5. INTERACTIVE CLI ---

def display_menu():
    """Displays the main CLI menu options."""
    print("\n--- Interactive Access Control System ---")
    print("1. Register New User (Automatic Secure Selection)")
    print("2. Authenticate User")
    print("3. View Registered Users (DEBUG)")
    print("4. Exit")
    print("-----------------------------------------")

def handle_registration():
    """Automatically processes credentials using all three methods and saves only Argon2."""
    print("\n--- New User Registration (Automatic Secure Selection) ---")
    username = input("Enter username: ").strip()
    if not username:
        print("Username cannot be empty.")
        return

    # --- START OF PASSWORD VALIDATION LOOP ---
    while True:
        print("\n--- Password Requirements ---")
        print("Password must be:")
        print(" - Minimum 8 characters long.")
        print(" - Contain at least one capital letter (A-Z).")
        print(" - Contain at least one small letter (a-z).")
        print(" - Contain at least one number (0-9).")
        print(" - Contain at least one special character (e.g., !, @, #, $).")
        print("-----------------------------")

        password = input("Enter password: ").strip()

        if not password:
            print("Password cannot be empty. Please try again.")
            continue

        if validate_password(password):
            break
        # Loop continues if validation fails

    # --- END OF PASSWORD VALIDATION LOOP ---

    methods_to_process = ['playfair', 'vigenere', 'argon2']
    processed_data = {} # Stores the raw output of all three encryptions

    print("\n--- Step 1: Automatic Credential Processing ---")

    # 1. Process all three methods and display results
    for method in methods_to_process:
        auth_handler = AUTH_METHODS[method]
        stored_data = auth_handler.hash(password)
        processed_data[method] = stored_data

        print(f"\nProcessing Method: {method.title()}")
        print(f"Security Level: {auth_handler.security_level}")

        # Display the result
        if method == 'argon2':
             print(f"  Hash Output: {stored_data[:40]}...")
        else:
             print(f"  Ciphertext Output: {stored_data}")

    print("\n--- Step 2: Security Assessment & Final Selection ---")
    print("The system has performed a security assessment on the processed credentials:")
    print(" - Playfair and Vigenere utilize INSECURE, REVERSIBLE ciphers.")
    print(" - Argon2 utilizes a SECURE, ONE-WAY key derivation function (hashing).")

    method_to_save = 'argon2'

    # 2. Clear existing methods and save ONLY the Argon2 credential
    if username in USER_DB:
        del USER_DB[username]

    argon2_data = processed_data[method_to_save]

    # Save the Argon2 credential
    if register_user(username, argon2_data, method_to_save):
         # Success message replacement
         print("success saving a user")

    # Final summary
    print("Registration process complete. 1 credential successfully saved (Argon2 only).")


def handle_authentication():
    """Handles the user input for authentication."""
    print("\n--- User Authentication ---")
    username = input("Enter username: ").strip()

    # CHECK 1: User existence check
    if username not in USER_DB:
        print(f"[AUTH FAILED] User '{username}' not found. User not found.")
        return

    # If user exists, prompt for password
    password = input("Enter password: ").strip()

    if not password:
        print("Password cannot be empty.")
        return

    authenticate_user(username, password)

def view_users():
    """Displays the current state of the database with streamlined output."""
    print("\n--- Registered User Database (DEBUG VIEW) ---")
    if not USER_DB:
        print("No users registered yet.")
        return

    print("user found")

    for username, user_methods in USER_DB.items():
        print(f"User: {username}")
        # Iterate over all methods stored for this user
        for method_key, stored_data in user_methods.items():
            # Only display Argon2 data as it's the only one saved
            if method_key == 'argon2':
                output_line = f"  Hashed Password: {stored_data}"
                print(output_line)


def main_menu():
    """The main interactive loop."""
    print("\n[SECURITY RESEARCHER APP] Welcome to the Access Control Demo.")
    print("----------------------------------------------------------------")

    while True:
        display_menu()
        choice = input("Enter your choice (1-4): ").strip()

        if choice == '1':
            handle_registration()
        elif choice == '2':
            handle_authentication()
        elif choice == '3':
            view_users()
        elif choice == '4':
            print("Exiting system. Goodbye!")
            sys.exit(0)
        else:
            print("Invalid choice. Please enter a number between 1 and 4.")

if __name__ == "__main__":
    main_menu()


[SECURITY RESEARCHER APP] Welcome to the Access Control Demo.
----------------------------------------------------------------

--- Interactive Access Control System ---
1. Register New User (Automatic Secure Selection)
2. Authenticate User
3. View Registered Users (DEBUG)
4. Exit
-----------------------------------------
Enter your choice (1-4): 1

--- New User Registration (Automatic Secure Selection) ---
Enter username: ahmed

--- Password Requirements ---
Password must be:
 - Minimum 8 characters long.
 - Contain at least one capital letter (A-Z).
 - Contain at least one small letter (a-z).
 - Contain at least one number (0-9).
 - Contain at least one special character (e.g., !, @, #, $).
-----------------------------
